In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
import scipy.misc
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
#import essentia, essentia.standard as ess

In [2]:
def chargen(char=20):
    print char
    char_gen = []
    i=0
    genre={}
    cont=0
    for folder in os.listdir("./genres"): #iterates all folders    

        folder="./genres/"+folder
        if not folder in genre.keys(): #creates a dictionary genre->number
            genre[folder]=i
            i+=1

        for filename in os.listdir(folder): #iterates all files in the folder
            filename = folder+"/"+filename

            if filename.endswith((".au",".mp3",".wav",".aiff")):
                audio, fs = librosa.load(filename, duration=29.0, mono=True, sr=22050) #loads the first 30 seconds of the song
                mfcc = librosa.feature.mfcc(audio, sr=fs, n_mfcc=char) #calculates MFCC of the song
                #The size of the matrix will be number_of_MFCCs*1292, which is a feature every 0.02s
                #By default, it's 20*1291

                """The first comlumn gives useless information, so we remove it
                Works as offset"""
                mfcc = np.delete(mfcc, 0, axis=1)
                #mfcc = np.ndarray.tolist(mfcc)

                char_gen.append((mfcc,genre[folder])) #adds the array to a list
                
                if cont%10==0:
                    print cont%100,
                
                cont+=1
        print 
                    
    return np.array(char_gen)

In [3]:
def char1():
    char_gen=[]
    i=0
    genre={}
    cont=0
    for folder in os.listdir("./genres"): #iterates all folders    

        folder="./genres/"+folder
        if not folder in genre.keys(): #creates a dictionary genre->number
            genre[folder]=i
            i+=1

        for filename in os.listdir(folder): #iterates all files in the folder
            filename = folder+"/"+filename

            if filename.endswith((".au",".mp3",".wav",".aiff")):
                audio, fs = librosa.load(filename, duration=29.0, mono=True, sr=22050) #loads the first 30 seconds of the song
                mfcc = librosa.feature.mfcc(audio, sr=fs) #calculates MFCC of the song
                #The size of the matrix will be number_of_MFCCs*1292, which is a feature every 0.02s
                #By default, it's 20*1291

                """The first comlumn gives useless information, so we remove it
                Works as offset"""
                mfcc = np.delete(mfcc, 0, axis=1)
                                
                #librosa.display.specshow(mfcc, x_axis='time')
                #plt.show()

                mfcc=np.transpose(mfcc)
                mfcc=mfcc.flatten() #converts the matrix into a 1D array

                char_gen.append((mfcc,genre[folder])) #adds the array to a list
                
                if cont%10==0:
                    print cont%100,
                
                cont+=1
                '''if cont>=10:
                    break'''
        print 
                    
    return np.array(char_gen)

In [4]:
def char2():
    char_gen=[]
    i=0
    genre={}
    for folder in os.listdir("./genres"): #iterates all folders    
        cont=0

        folder="./genres/"+folder
        if not folder in genre.keys(): #creates a dictionary genre->number
            genre[folder]=i
            i+=1

        for filename in os.listdir(folder): #iterates all files in the folder
            filename = folder+"/"+filename

            if filename.endswith((".au",".mp3",".wav",".aiff")):
                audio, fs = librosa.load(filename, duration=30.0, mono=True, sr=22050) #loads the first 30 seconds of the song
                mfcc = librosa.feature.mfcc(audio, sr=fs, n_fft=1024, hop_length=30*22050) #calculates MFCC of the song
                #The size of the matrix will be number_of_MFCCs*1292, which is a feature every 0.02s
                #By default, it's 20*1922

                """The first comlumn gives useless information, so we remove it
                Works as offset"""
                mfcc = np.delete(mfcc, 0, axis=1)
                
                #librosa.display.specshow(mfcc, x_axis='time')
                #plt.show()
                
                mfcc=mfcc.flatten() #converts the matrix into a 1D array

                char_gen.append((mfcc,genre[folder])) #adds the array to a list

                cont+=1
                if cont>=1:
                    break
                
    return char_gen

In [5]:
def char3(char=5):
    print char
    char_gen=[]
    maxs=[]
    mins=[]
    i=0
    genre={}
    cont=0
    scaler = StandardScaler()
    for folder in os.listdir("./genres"): #iterates all folders    

        folder="./genres/"+folder
        if not folder in genre.keys(): #creates a dictionary genre->number
            genre[folder]=i
            i+=1

        print str(cont/100)+":",    
            
        for filename in os.listdir(folder): #iterates all files in the folder
            filename = folder+"/"+filename

            if filename.endswith((".au",".mp3",".wav",".aiff")):
                audio, fs = librosa.load(filename, duration=29.0, mono=True, sr=22050) #loads the first 30 seconds of the song
                mfcc = librosa.feature.mfcc(audio, sr=fs, n_mfcc=char) #calculates MFCC of the song
                #The size of the matrix will be number_of_MFCCs*1292, which is a feature every 0.02s
                #By default, it's 20*1291

                """The first comlumn gives useless information, so we remove it
                Works as offset"""
                mfcc = np.delete(mfcc, 0, axis=1)
                
                #mfcc = scaler.fit_transform(mfcc)
                                
                maxs.append(np.amax(mfcc))
                mins.append(np.amin(mfcc))
                
                #mfcc=np.transpose(mfcc)
                #mfcc=mfcc.flatten() #converts the matrix into a 1D array

                char_gen.append((mfcc,genre[folder])) #adds the array to a list
                
                if cont%10==0:
                    print cont%100,
                
                cont+=1
        print 
    step = (max(maxs) - min(mins))/100
    return np.array(char_gen), step

In [6]:
def histogram(step, char, amount):
    histogram = np.zeros(amount)
    for c in char:
        histogram[int(c/step)] += 1.
    histogram = histogram/len(char)
    histogram = np.ndarray.tolist(histogram)
    return histogram

In [7]:
def reshape_pca(char):
    mfcc = np.array(char[:,0 ])
    mfcc = np.concatenate(mfcc).reshape(1000,24960)
    pca=PCA(n_components = 100, svd_solver='full')
    pca.fit(mfcc)
    mfcc=pca.transform(mfcc)
    return mfcc

In [8]:
def train_test(mfcc, char, n_set):
    train_mfcc, test_mfcc=[],[]
    train_genre, test_genre=[], []

    for i in range(1000):
        if (i+n_set)%10==0:
            test_mfcc.append(mfcc[i])
            test_genre.append(char[i][1])
        else:
            train_mfcc.append(mfcc[i])
            train_genre.append(char[i][1])

    train_mfcc = np.array(train_mfcc)
    test_mfcc = np.array(test_mfcc)
    train_genre = np.array(train_genre)
    test_genre = np.array(test_genre)
    return train_mfcc, train_genre, test_mfcc, test_genre

In [9]:
def classify(train_mfcc, train_genre, test_mfcc, test_genre):
    success=0
    clf=SVC(C=0.0001)
    clf.fit(train_mfcc,train_genre)
    for i in range(100):
        test=test_mfcc[i].reshape(1, -1)
        pred = clf.predict(test)
        if pred[0]==test_genre[i]:
            success+=1
    return success

In [10]:
def all_hist(char, step, amount):
    hist=[]
    temp_char=[]
    for c in char:
        temp = histogram(step, c[0], amount)
        hist.append((temp, c[1]))
        temp_char.append(temp)
    return hist, temp_char


In [11]:
def more_hist(char, step, amount):
    hist = []
    temp_char=[]
    for c in char:
        tmp = c[0]
        temp = []
        for v in tmp:
            temp+=histogram(step, v, amount)
        temp_char.append(temp)
        hist.append((temp, c[1]))
    return hist, temp_char    

In [12]:
def mode1():
    char = char1()
    mfcc = reshape_pca(char)
    train_mfcc, train_genre, test_mfcc, test_genre = train_test(mfcc, char)
    classify(train_mfcc, train_genre, test_mfcc, test_genre)

In [13]:
def mode2():
    char = char2()
    mfcc = reshape_pca(char)
    train_mfcc, train_genre, test_mfcc, test_genre = train_test(mfcc, char)
    classify(train_mfcc, train_genre, test_mfcc, test_genre)

In [14]:
def mode3(char, step, amount):
    hist, char = all_hist(char, step, amount)
    train_mfcc, train_genre, test_mfcc, test_genre = train_test(char, hist)
    return classify(train_mfcc, train_genre, test_mfcc, test_genre)

In [15]:
def mode4(char, step, amount):
    hist, char = more_hist(char, step, amount)
    train_mfcc, train_genre, test_mfcc, test_genre = train_test(char, hist)
    return classify(train_mfcc, train_genre, test_mfcc, test_genre)

In [36]:
for i in range(5,21,5):
    char_gen = chargen(i)
    filename = "values"+str(i)
    np.save(filename, char_gen)

 5
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
10
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
15
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
20
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 10 20 30 40 50 60 70 80 90
0 

In [49]:
for i in range(5,6,5):
    filename = "values"+str(i)+".npy"
    char = np.load(filename)
    #print char.shape, char[0].shape, char[0][0].shape, char[0][0][0].shape
    #step = (np.amax(np.amax(np.amax(char))) - np.amin(np.amin(np.amin(char))))/100

In [74]:
char, step = char3(5)

0: 0 10 20 30 40 50 60 70 80 90
1: 0 10 20 30 40 50 60 70 80 90
2: 0 10 20 30 40 50 60 70 80 90
3: 0 10 20 30 40 50 60 70 80 90
4: 0 10 20 30 40 50 60 70 80 90
5: 0 10 20 30 40 50 60 70 80 90
6: 0 10 20 30 40 50 60 70 80 90
7: 0 10 20 30 40 50 60 70 80 90
8: 0 10 20 30 40 50 60 70 80 90
9: 0 10 20 30 40 50 60 70 80 90


In [54]:
hist, charco = more_hist(char,(step*100)/50, 50)

([array([0.03044872, 0.00080128, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.00240385, 0.00641026, 0.02083333,
       0.03125   , 0.06410256, 0.11939103, 0.11378205, 0.15865385,
       0.14663462, 0.10416667, 0.08333333, 0.0713141 , 0.04647436]), array([0.        , 0.        , 0.00160256, 0.00961538, 0.04407051,
       0.24759615, 0.43589744, 0.21794872, 0.04326923, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0. 

In [57]:
results=[]
for i in range(1,100):
    if i%10==0: print i,
    results.append((i, mode3(char, (step*100)/i, amount=i)))

print
results.sort(key=lambda x:x[1], reverse=True)
print results[0]

TypeError: only size-1 arrays can be converted to Python scalars

In [96]:
results=[]
for i in range(5,100):
    if i%10==0: print i,
    results.append((i, mode4(char, (step*100)/i, amount=i)))

print
results.sort(key=lambda x:x[1], reverse=True)
print results
print results[0]

10 20 30 40 50 60 70 80 90
[(17, 39), (18, 38), (16, 37), (20, 37), (21, 37), (39, 36), (44, 36), (52, 36), (13, 35), (19, 35), (23, 35), (37, 35), (40, 35), (45, 35), (46, 35), (47, 35), (51, 35), (53, 35), (57, 35), (58, 35), (59, 35), (60, 35), (15, 34), (36, 34), (38, 34), (41, 34), (42, 34), (43, 34), (48, 34), (49, 34), (50, 34), (54, 34), (56, 34), (61, 34), (62, 34), (14, 33), (22, 33), (24, 33), (25, 33), (30, 33), (31, 33), (34, 33), (35, 33), (55, 33), (63, 33), (64, 33), (65, 33), (81, 33), (82, 33), (84, 33), (85, 33), (89, 33), (93, 33), (32, 32), (66, 32), (67, 32), (68, 32), (69, 32), (71, 32), (80, 32), (83, 32), (86, 32), (87, 32), (88, 32), (90, 32), (91, 32), (92, 32), (94, 32), (95, 32), (96, 32), (97, 32), (98, 32), (99, 32), (33, 31), (70, 31), (72, 31), (73, 31), (74, 31), (75, 31), (76, 31), (77, 31), (78, 31), (79, 31), (11, 30), (27, 30), (29, 30), (12, 29), (26, 29), (9, 28), (28, 28), (7, 27), (8, 26), (10, 24), (6, 22), (5, 19)]
(17, 39)


In [16]:
char, step = char3(15)

15
0: 0 10 20 30 40 50 60 70 80 90
1: 0 10 20 30 40 50 60 70 80 90
2: 0 10 20 30 40 50 60 70 80 90
3: 0 10 20 30 40 50 60 70 80 90
4: 0 10 20 30 40 50 60 70 80 90
5: 0 10 20 30 40 50 60 70 80 90
6: 0 10 20 30 40 50 60 70 80 90
7: 0 10 20 30 40 50 60 70 80 90
8: 0 10 20 30 40 50 60 70 80 90
9: 0 10 20 30 40 50 60 70 80 90


In [23]:
#char, step = char3(15)

def mode5(char, step, amount, n_set, gamma):
    hist, char = more_hist(char, step, amount)
    train_mfcc, train_genre, test_mfcc, test_genre = train_test(char, hist, n_set)
    return classify2(train_mfcc, train_genre, test_mfcc, test_genre, gamma)

def classify2(train_mfcc, train_genre, test_mfcc, test_genre, g):
    success=0
    succ_gen = np.zeros(10)
    clf=SVC(kernel = 'rbf', gamma=g)
    clf.fit(train_mfcc,train_genre)
    for i in range(100):
        test=test_mfcc[i].reshape(1, -1)
        pred = clf.predict(test)
        if pred[0]==test_genre[i]:
            succ_gen[test_genre[i]] += 1
            success+=1
    print succ_gen, np.mean(succ_gen), success
    return success


'''for j in range(-9,10):
    print "gamma:", 1.4+(j/100.)'''
for j in range(10):
    print "set:", j
    for i in range(87,93):
        print str(i),":",
        mode5(char, (step*100)/i, amount=i, n_set=j, gamma=1.4)

 set: 0
87 : [6. 9. 6. 7. 4. 7. 6. 7. 5. 4.] 6.1 61
88 : [7. 9. 6. 7. 4. 7. 6. 7. 5. 4.] 6.2 62
89 : [7. 9. 6. 7. 4. 7. 6. 7. 5. 4.] 6.2 62
90 : [6. 9. 6. 7. 4. 7. 6. 7. 5. 4.] 6.1 61
91 : [5. 9. 6. 7. 4. 7. 6. 7. 5. 4.] 6.0 60
92 : [5. 9. 6. 7. 4. 7. 6. 7. 5. 4.] 6.0 60
set: 1
87 : [ 7.  8.  6.  4.  8.  7. 10.  8.  3.  6.] 6.7 67
88 : [ 7.  8.  6.  4.  9.  7. 10.  8.  3.  6.] 6.8 68
89 : [ 7.  8.  6.  4.  9.  7. 10.  8.  3.  6.] 6.8 68
90 : [ 7.  8.  7.  4.  9.  7. 10.  8.  3.  6.] 6.9 69
91 : [ 7.  8.  7.  4.  9.  7. 10.  8.  3.  6.] 6.9 69
92 : [ 7.  8.  7.  5.  9.  7. 10.  8.  3.  6.] 7.0 70
set: 2
87 : [ 6. 10.  7.  7.  8.  9.  6. 10.  8.  3.] 7.4 74
88 : [ 6. 10.  7.  7.  8.  9.  6. 10.  8.  2.] 7.3 73
89 : [ 6. 10.  7.  7.  8.  9.  6. 10.  8.  2.] 7.3 73
90 : [ 6. 10.  7.  7.  8.  9.  6. 10.  8.  3.] 7.4 74
91 : [ 6. 10.  7.  7.  8.  9.  6. 10.  8.  3.] 7.4 74
92 : [ 6. 10.  7.  7.  8.  9.  6. 10.  8.  2.] 7.3 73
set: 3
87 : [ 5. 10.  6.  4.  5.  7. 10.  9.  5.  3.] 6.4 64
88 : 

In [34]:
#leave one out
from sklearn.model_selection import LeaveOneOut
'''hist, char2 = more_hist(char, step, 90)

loo = LeaveOneOut()
loo.get_n_splits(char2)'''

i = 0
success = 0

for train_index, test_index in loo.split(char2):
    X_train = np.array([char2[j] for j in train_index])
    X_test = np.array([char2[j] for j in test_index])
    y_train  = np.array([hist[j][1] for j in train_index])
    y_test = np.array([hist[j][1] for j in test_index])
    clf=SVC(kernel = 'linear')
    clf.fit(X_train,y_train)
    test=X_test.reshape(1, -1)
    pred = clf.predict(X_test)
    if pred[0]==y_test:
        success+=1
    if i%10==0:
        print str(i),
    i+=1

print success

0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 630 640 650 660 670 680 690 700 710 720 730 740 750 760 770 780 790 800 810 820 830 840 850 860 870 880 890 900 910 920 930 940 950 960 970 980 990 610


In [17]:
import numpy as np
import matplotlib.pyplot as plt
import operator

from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier

def mode6(char, step, amount, n_set):
    hist, char = more_hist(char, step, amount)
    train_mfcc, train_genre, test_mfcc, test_genre = train_test(char, hist, n_set)
    return classify3(train_mfcc, train_genre, test_mfcc, test_genre)

def classify3(train_mfcc, train_genre, test_mfcc, test_genre):
    value = dict([(key, 0) for key in range(15)])
    order = dict([(key, 0) for key in range(15)])
    forest = ExtraTreesClassifier()
    forest.fit(train_mfcc,train_genre)
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
    indices = np.argsort(importances)[::-1]
    # Print the feature ranking
    print("Feature ranking:")

    for f in range(train_mfcc.shape[1]):
        #print("%d. feature %d (%f)" % (f + 1, indices[f]/90, importances[indices[f]]))
        if not importances[indices[f]]<=10**-10:
            print("%d. feature %d (%f)" % (f + 1, indices[f]/90, importances[indices[f]]))
            value[indices[f]/90] +=  1
        else:
            break
    
    value = sorted(value.items(), key=operator.itemgetter(1))
    
    print value

    # Plot the feature importances of the forest
    '''plt.figure()
    plt.title("Feature importances")
    plt.bar(range(100), importances[indices],
           color="r", yerr=std[indices], align="center")
    plt.xticks(range(100), indices)
    plt.xlim([-1, 10])
    plt.show()'''

mode6(char, (step*100)/90, amount=90, n_set=4)

Feature ranking:
1. feature 1 (0.013891)
2. feature 7 (0.011511)
3. feature 12 (0.010749)
4. feature 3 (0.010640)
5. feature 3 (0.010156)
6. feature 3 (0.010020)
7. feature 8 (0.009884)
8. feature 5 (0.009602)
9. feature 5 (0.009223)
10. feature 7 (0.009086)
11. feature 2 (0.009045)
12. feature 4 (0.009021)
13. feature 9 (0.008805)
14. feature 0 (0.008585)
15. feature 0 (0.008529)
16. feature 0 (0.008526)
17. feature 9 (0.008498)
18. feature 10 (0.008472)
19. feature 2 (0.008257)
20. feature 0 (0.008175)
21. feature 4 (0.008113)
22. feature 2 (0.008004)
23. feature 5 (0.007964)
24. feature 8 (0.007883)
25. feature 0 (0.007778)
26. feature 3 (0.007567)
27. feature 0 (0.007484)
28. feature 0 (0.007407)
29. feature 2 (0.007307)
30. feature 3 (0.007070)
31. feature 3 (0.006990)
32. feature 10 (0.006947)
33. feature 1 (0.006834)
34. feature 1 (0.006820)
35. feature 0 (0.006792)
36. feature 8 (0.006747)
37. feature 0 (0.006713)
38. feature 0 (0.006689)
39. feature 3 (0.006618)
40. feature 0 

In [28]:
from sklearn.naive_bayes import GaussianNB

def mode7(char, step, amount, n_set):
    hist, char = more_hist(char, step, amount)
    train_mfcc, train_genre, test_mfcc, test_genre = train_test(char, hist, n_set)
    return classify4(train_mfcc, train_genre, test_mfcc, test_genre)

def classify4(train_mfcc, train_genre, test_mfcc, test_genre):
    success=0
    succ_gen = np.zeros(10)
    gnb = GaussianNB()
    y_pred = gnb.fit(train_mfcc,train_genre).predict(test_mfcc)
    #print test_genre == y_pred
    print("Number of mislabeled points out of a total %d points : %d"
      % (test_mfcc.shape[0],(test_genre != y_pred).sum()))

mode7(char, (step*100)/90, amount=90, n_set=4)

Number of mislabeled points out of a total 100 points : 56


In [32]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score

def mode8(char, step, amount, n_set):
    hist, char = more_hist(char, step, amount)
    train_mfcc, train_genre, test_mfcc, test_genre = train_test(char, hist, n_set)
    return classify5(train_mfcc, train_genre, test_mfcc, test_genre)

def classify5(train_mfcc, train_genre, test_mfcc, test_genre):
    clf = AdaBoostClassifier(n_estimators=100)
    scores = cross_val_score(clf, train_mfcc, train_genre)
    print scores.mean()

mode8(char, (step*100)/90, amount=90, n_set=4)  

0.18111111111111108


# Results
* char = 5
* step = 18

* **result** = 36


* char = 20
* step = 18

* **result** = 37


* char = 12
* step = 18

* **result** = 37